In [ ]:
!pip install pandas pyarrow matplotlib seaborn

In [ ]:
!pip install scikit-learn xgboost

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pyarrow.parquet as pq

class DataLoader:
    def __init__(self, path_prefix, batch_size):
        self.path_prefix = path_prefix
        self.batch_size = batch_size

    def load_data(self, columns=None):
        dataset = pq.ParquetDataset(self.path_prefix + 'full_df.parquet', use_legacy_dataset=False)
        table = dataset.read(columns=columns)
        for i in range(0, table.num_rows, self.batch_size):
            yield table.slice(i, self.batch_size).to_pandas()

    def preprocess_data(self, df):
        df = df.copy()
        df = self.encode_and_extend_df(df)
        df = self.drop_columns(df)
        df = self.calculate_additional_features(df)
        df.fillna(0, inplace=True)
        return df

    def encode_and_extend_df(self, df):
        time_of_day_mapping = {'Morning': 0, 'Afternoon': 1, 'Evening': 2, 'Late night': 3}
        if 'pickup_timeofday' in df.columns and 'dropoff_timeofday' in df.columns:
            df['pickup_timeofday_encoded'] = df['pickup_timeofday'].map(time_of_day_mapping)
            df['dropoff_timeofday_encoded'] = df['dropoff_timeofday'].map(time_of_day_mapping)
        else:
            raise ValueError("DataFrame is missing necessary time of day columns.")
        return df

    def drop_columns(self, df):
        columns_to_drop = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'store_and_fwd_flag',
                           'pickup_timeofday', 'dropoff_timeofday', 'pickup_day', 'dropoff_day',
                           'pickup_datetime', 'dropoff_datetime']
        df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
        return df

    def calculate_additional_features(self, df):
        df = df[df['trip_distance'] != 0.0].copy()
        df['trip_duration_hours'] = df['trip_duration'] / 60
        df['speed_mph'] = df.apply(
            lambda row: row['trip_distance'] / row['trip_duration_hours'] if row['trip_duration_hours'] > 0 else 0, axis=1
        )
        return df

class BaseModel:
    def __init__(self, model):
        self.model = model

    def split_data(self, df, target_col='trip_duration', test_size=0.2, val_size=0.25):
        train_val, test = train_test_split(df, test_size=test_size, random_state=42)
        train, val = train_test_split(train_val, test_size=val_size, random_state=42)
        return train, val, test

    def train(self, X_train, y_train, batch_mode=False):
        if batch_mode:
            for X_batch, y_batch in zip(X_train, y_train):
                self.model.partial_fit(X_batch, y_batch)
        else:
            self.model.fit(X_train, y_train)

    def predict(self, X):
        return self.model.predict(X)

    def evaluate(self, y_true, y_pred):
        return np.sqrt(mean_squared_error(y_true, y_pred))

    def train_model(self, X_train, y_train, X_val, y_val):
        self.train(X_train, y_train)
        y_val_pred = self.predict(X_val)
        val_rmse = self.evaluate(y_val, y_val_pred)
        return val_rmse

    def test_model(self, X_test, y_test):
        y_test_pred = self.predict(X_test)
        test_rmse = self.evaluate(y_test, y_test_pred)
        return test_rmse

class LinearRegressionModel(BaseModel):
    def __init__(self):
        super().__init__(LinearRegression())

class XGBoostModel(BaseModel):
    def __init__(self):
        super().__init__(xgb.XGBRegressor(
            objective='reg:squarederror',
            eval_metric='rmse',
            verbosity=0
        ))

def main():
    data_loader = DataLoader('/content/drive/Shareddrives/Data_Mining_Project_NYC/nyc_dataset/all_months/processed_dataset/', batch_size=100000)
    columns = ['trip_duration', 'trip_distance', 'pickup_timeofday', 'dropoff_timeofday']
    epochs = 5

    linear_regression_model = LinearRegressionModel()
    xgboost_model = XGBoostModel()

    val_rmses_lr = []
    val_rmses_xgb = []
    for df_batch in data_loader.load_data(columns=columns):
            processed_df = data_loader.preprocess_data(df_batch)
            train_df, val_df, test_df = linear_regression_model.split_data(processed_df)

            target_col = 'trip_duration'
            X_train, y_train = train_df.drop(target_col, axis=1), train_df[target_col]
            X_val, y_val = val_df.drop(target_col, axis=1), val_df[target_col]

            val_rmse_lr = linear_regression_model.train_model(X_train, y_train, X_val, y_val)
            val_rmse_xgb = xgboost_model.train_model(X_train, y_train, X_val, y_val)

            val_rmses_lr.append(val_rmse_lr)
            val_rmses_xgb.append(val_rmse_xgb)
            print(f'batch:{df_batch}; XGBoost Validation RMSE: {val_rmse_xgb}, Linear Regression Validation RMSE: {val_rmse_lr}')
    avg_val_rmse_lr = np.mean(val_rmses_lr)
    avg_val_rmse_xgb = np.mean(val_rmses_xgb)

    print(f'XGBoost Validation RMSE: {avg_val_rmse_xgb}, Linear Regression Validation RMSE: {avg_val_rmse_lr}')

    X_test, y_test = test_df.drop(target_col, axis=1), test_df[target_col]
    test_rmse_lr = linear_regression_model.test_model(X_test, y_test)
    test_rmse_xgb = xgboost_model.test_model(X_test, y_test)

    print(f'Linear Regression Test RMSE: {test_rmse_lr}')
    print(f'XGBoost Test RMSE: {test_rmse_xgb}')

if __name__ == "__main__":
    main()


Streaming output truncated to the last 5000 lines.
0           4.616667           0.50          Evening           Evening
1          11.433333           1.79          Evening           Evening
2           7.316667           0.90          Evening           Evening
3          10.783333           1.90          Evening           Evening
4          21.916667           4.40          Evening           Evening
...              ...            ...              ...               ...
99995      16.033333           1.51          Evening           Evening
99996      15.766667           3.21          Evening           Evening
99997      19.116667           1.61          Evening           Evening
99998      17.300000           1.74          Evening           Evening
99999      39.600000          15.67          Evening           Evening

[100000 rows x 4 columns]; XGBoost Validation RMSE: 27.74382737138723, Linear Regression Validation RMSE: 4.625470566563056e-14
batch:       trip_duration  trip_distan